In [ ]:
#List files

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Load data

df = pd.read_csv("./adult.data", header=None)
columns = [
    'age', # continuous.
    'workclass', # Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
    'fnlwgt', # continuous. final weight based on Gov CPS data
    'education', # Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
    'education-num', # continuous.
    'marital-status', # Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
    'occupation', # Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
    'relationship', # Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
    'race', # White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
    'sex', # Female, Male.
    'capital-gain', # continuous.
    'capital-loss', # continuous.
    'hours-per-week', # continuous.
    'native-country', # United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
    'income'
]
df.columns = columns

# df.columns = df.columns.str.lower()

with pd.option_context('display.max_rows', 6, 'display.max_columns', None): 
    display(df)    


In [ ]:
# Prepare columns

categorical_columns = [
    'workclass', # Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
    'education', # Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
    'marital-status', # Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
    'occupation', # Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
    'relationship', # Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
    'race', # White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
    'sex', # Female, Male.
    'native-country', # United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
]
numerical_columns = [
    'age', # continuous.
    'fnlwgt', # continuous. final weight based on Gov CPS data
    'education-num', # continuous.
    'capital-gain', # continuous.
    'capital-loss', # continuous.
    'hours-per-week', # continuous.
]

y_column = 'income'

df.info()

In [ ]:
#Check data

#trim all categorical fields
for column in categorical_columns:
    df[column] = df[column].str.strip()

isnull_count = pd.DataFrame(df.isnull().sum(), columns=["isnull count"])
isna_count = pd.DataFrame(df.isna().sum(), columns=["isna count"])
isin_count = pd.DataFrame(df.isin(["NaN", "-", "", ".", "_", "?"]).sum(), columns=["isin count"])
pd.concat([isnull_count, isna_count, isin_count], axis=1)



In [ ]:
# Cleanup data
df_clean = df[categorical_columns+numerical_columns+[y_column]].copy()

for column in categorical_columns:
    df_clean[column].replace({"?": "unknown"}, inplace=True)

# prepare y
df_clean['low_income'] = df.income.str.strip() == '<=50K'
del df_clean["income"]

# delete rows with unknown fields
for column in categorical_columns:
    df_clean[column].replace({"unknown": np.nan}, inplace=True)
df_clean = df_clean.dropna()    

#reset index
df_clean = df_clean.reset_index(drop=True)

df_clean

In [ ]:
# EDA Exploritory Data Analysis

with pd.option_context('display.max_rows', 6, 'display.max_columns', None): 
    
    df_selection_by_sex = df_clean[df_clean.age >= 40].groupby("sex").low_income.agg(["mean", "count"]).T
    display("Low income people working fulltime by sex:", df_selection_by_sex)
    
    df_selection_by_age = df_clean[df_clean.age >= 40].groupby("age").low_income.agg(["mean", "count"]).T
    display("Low income people working fulltime by age:", df_selection_by_age)
    
    df_selection_by_education = df_clean[df_clean.age >= 40].groupby("education").low_income.agg(["mean", "count"]).sort_values(by=['mean']).T
    display("Low income people working fulltime by education:", df_selection_by_education)
    
    df_selection_by_country = df_clean[df_clean.age >= 40].groupby("native-country").low_income.agg(["mean", "count"]).sort_values(by=['mean']).T
    display("Low income people working fulltime by country:", df_selection_by_country)
    
    
    
